In [233]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [234]:
'''To read data from diamonds.csv'''
headers = ["carat",	"cut","color","clarity","depth","table","price","x","y","z"]
data = pd.read_csv('Regression_Diamonds_data/diamonds.csv', na_values=np.NaN,    
         header=None,  names = headers) 
data = data.reset_index(drop=True)
data = data.iloc[1:]
# data = data[:1000]
print(data.describe())
print(data.head()) 

        carat    cut  color clarity  depth  table  price      x      y      z
count   53940  53940  53940   53940  53940  53940  53940  53940  53940  53940
unique    273      5      7       8    184    127  11602    554    552    375
top       0.3  Ideal      G     SI1     62     56    605   4.37   4.34    2.7
freq     2604  21551  11292   13065   2239   9881    132    448    437    767
  carat      cut color clarity depth table price     x     y     z
1  0.23    Ideal     E     SI2  61.5    55   326  3.95  3.98  2.43
2  0.21  Premium     E     SI1  59.8    61   326  3.89  3.84  2.31
3  0.23     Good     E     VS1  56.9    65   327  4.05  4.07  2.31
4  0.29  Premium     I     VS2  62.4    58   334   4.2  4.23  2.63
5  0.31     Good     J     SI2  63.3    58   335  4.34  4.35  2.75


In [235]:
'''Check for NaNs'''
for h in headers:
    print(h,"NaN :",data[h].isnull().sum())
    


carat NaN : 0
cut NaN : 0
color NaN : 0
clarity NaN : 0
depth NaN : 0
table NaN : 0
price NaN : 0
x NaN : 0
y NaN : 0
z NaN : 0


In [236]:
'''Convert categorical attributes to numeric'''
cat_to_num = {"cut":{"Fair":0, "Good":1, "Very Good":2, "Premium":3, "Ideal":4}, 
              "clarity":{"I1":0, "SI2":1, "SI1":2, "VS2":3, "VS1":4, "VVS2":5, "VVS1":6, "IF":7},
              "color":{"J":0, "I":1, "H":2, "G":3, "F":4, "E":5, "D":6}}
data = data.replace(cat_to_num)
print(data.head())
# data = pd.get_dummies(data,columns=['color'])

  carat  cut  color  clarity depth table price     x     y     z
1  0.23    4      5        1  61.5    55   326  3.95  3.98  2.43
2  0.21    3      5        2  59.8    61   326  3.89  3.84  2.31
3  0.23    1      5        4  56.9    65   327  4.05  4.07  2.31
4  0.29    3      1        3  62.4    58   334   4.2  4.23  2.63
5  0.31    1      0        1  63.3    58   335  4.34  4.35  2.75


In [237]:
for h in headers:
    data[h] = pd.to_numeric(data[h],downcast='float')

# data

In [238]:
'''Separating X & y'''
X = data.drop('price', axis=1)
y = data.price
print(X.head())
print(y.head())

   carat  cut  color  clarity      depth  table     x     y     z
1   0.23  4.0    5.0      1.0  61.500000   55.0  3.95  3.98  2.43
2   0.21  3.0    5.0      2.0  59.799999   61.0  3.89  3.84  2.31
3   0.23  1.0    5.0      4.0  56.900002   65.0  4.05  4.07  2.31
4   0.29  3.0    1.0      3.0  62.400002   58.0  4.20  4.23  2.63
5   0.31  1.0    0.0      1.0  63.299999   58.0  4.34  4.35  2.75
1    326.0
2    326.0
3    327.0
4    334.0
5    335.0
Name: price, dtype: float32


In [239]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=1)

### add standardize experiment

In [240]:
'''Minkowski distance function'''
def minkowski_dist(test_pt, train_X, p=1):
    test_pt = np.array(test_pt)
    train_X = np.array(train_X)
    temp = abs(test_pt - train_X) 
    if p==2:
        temp = temp*temp
        temp = np.sum(np.sqrt(temp),axis=1)
        return temp
    if p==1:
        return np.sum(temp,axis=1)
    if p==0:
        temp = temp!=0
        temp = temp*1
        return np.sum(temp,axis=1)
# def minkowski_dist(x1, x2, p=1):
#     dim = len(x1)
#     dist = 0
#     for d in range(dim):
# #         print(x1[d],type(x1[d]))
#         dist += abs(x1[d] - x2[d])**p
#     dist = dist**(1/p)
#     return dist
# print(X_test.iloc[0])
# d = minkowski_dist(X_test.iloc[0],X,0)
# print("d",d.shape,d.dtype)
# print(d)

In [241]:
all_dists = []  #only 25 smallest distances
# def cal_all_dists(train_X,test_X):
#     c=1;
#     for i in test_X.index:
#         cur_dists = []
# #         print(X_test.loc[i])
#         for j in train_X.index:
#             cur_dists.append(minkowski_dist(test_X.loc[i],train_X.loc[j]))
#         df_cur_dists = pd.DataFrame(data=cur_dists, columns=['dist'], index=y_train.index)
#         df_sorted_curdists = df_cur_dists.sort_values(by=['dist'], axis=0)
#         all_dists.append(df_sorted_curdists)
# #         print("done",c)
#         c += 1;

def cal_all_dists(train_X,test_X):
    max_nn = 25
    train_Xnp = np.array(train_X)
    test_X = np.array(test_X)
    for i in range(test_X.shape[0]):
        cur_dists = minkowski_dist(test_X[i],train_Xnp,2)
#         print(cur_dists.shape)
#         cur_dists = cur_dists[np.argpartition(cur_dists, 25)[:25]]
        idx = np.argpartition(cur_dists, max_nn)[:max_nn]
        cur_dists = cur_dists[idx]
        idx_df = [train_X.iloc[j].name for j in idx]
        df_cur_dists = pd.DataFrame(data=cur_dists, columns=['dist'], index=idx_df)
        df_cur_dists = df_cur_dists.sort_values(by=['dist'], axis=0)
        all_dists.append(df_cur_dists)

cal_all_dists(X_train,X_test)
print(len(all_dists))
# print(all_dists)


/home/mansi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in sqrt
  


10788


In [242]:
# def cal_simple_mean(val_list,k):
    
'''criterion = simple_mean, weighted_mean'''
def knn_predict(test_X,k,criterion):
    criteria = ['simple_mean','weighted_mean']
    predictions = []
    c=0
    for i in test_X.index:
        nn_dists = all_dists[c][:k]
#         print(i,nn_dists)
        if(criterion==criteria[0]):
            sum_knn = sum([y_train.loc[j] for j in nn_dists.index])
            mean_knn = sum_knn/k
#             print(mean_knn)
            predictions.append(mean_knn)
        c+=1
    return predictions




In [243]:
y_pred = knn_predict(X_test,2,"simple_mean")
rsq_score = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse_score = mean_squared_error(y_test, y_pred)
print("K:2, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1")
print("R2 Score:",rsq_score)
print("MAE Score:",mae_score)
print("MSE Score:",mse_score)

y_pred = knn_predict(X_test,3,"simple_mean")
rsq_score = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse_score = mean_squared_error(y_test, y_pred)
print("K:3, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1")
print("R2 Score:",rsq_score)
print("MAE Score:",mae_score)
print("MSE Score:",mse_score)

y_pred = knn_predict(X_test,5,"simple_mean")
rsq_score = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse_score = mean_squared_error(y_test, y_pred)
print("K:5, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1")
print("R2 Score:",rsq_score)
print("MAE Score:",mae_score)
print("MSE Score:",mse_score)

y_pred = knn_predict(X_test,7,"simple_mean")
rsq_score = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse_score = mean_squared_error(y_test, y_pred)
print("K:7, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1")
print("R2 Score:",rsq_score)
print("MAE Score:",mae_score)
print("MSE Score:",mse_score)

y_pred = knn_predict(X_test,11,"simple_mean")
rsq_score = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse_score = mean_squared_error(y_test, y_pred)
print("K:11, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1")
print("R2 Score:",rsq_score)
print("MAE Score:",mae_score)
print("MSE Score:",mse_score)

y_pred = knn_predict(X_test,16,"simple_mean")
rsq_score = r2_score(y_test, y_pred)
mae_score = mean_absolute_error(y_test, y_pred)
mse_score = mean_squared_error(y_test, y_pred)
print("K:16, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1")
print("R2 Score:",rsq_score)
print("MAE Score:",mae_score)
print("MSE Score:",mse_score)

K:2, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1
R2 Score: 0.9598989453410379
MAE Score: 413.5690582128291
MSE Score: 623110.6926214312
K:3, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1
R2 Score: 0.962912579205125
MAE Score: 398.0481090100111
MSE Score: 576283.3086454084
K:5, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1
R2 Score: 0.9655328482097866
MAE Score: 383.7190767519466
MSE Score: 535568.2289988877
K:7, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1
R2 Score: 0.9669059905076641
MAE Score: 378.635719582605
MSE Score: 514231.64182992437
K:11, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1
R2 Score: 0.967017486242308
MAE Score: 376.65060504938145
MSE Score: 512499.16409264045
K:16, first 1000 rows, simple_mean, non-std, non-norm, split:80/20, split-random:1
R2 Score: 0.9668960573776264
MAE Score: 376.8716745457916
MSE Score: 51

In [ ]:
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

In [211]:
# print(X_test)
print(type(X_test.iloc[1].name))
print(X_test.index)
# print(all_dists[0].head())
# for i in X_test.index:
#     print(c,i)
# print(y_test.loc[508])
# print(y_train)
# np.array(y_test)

<class 'numpy.int64'>
Int64Index([508, 819, 453, 369, 243, 930, 263, 811, 319,  50,
            ...
            700, 827, 234, 585, 975, 431, 875, 551, 609, 208],
           dtype='int64', length=200)
